## Import libraries

In [19]:
import numpy as np
import pandas as pd


---------------------------------------------------------------------------------------------

# 1st STEP: SLICING SUPERSET

## Reading two dasets with 227 countries which repeat ciclycally for 5 different attributes:
### We have 5 datasets inside of one 'SUPER'dataset*. *Need to split them up in five separately*
### total
### renewable
### fossil fuel
---------------------------------------------------------------------------------------------


In [20]:
capacity = pd.read_csv('../raw_data/electricity_capacity.csv',sep=',',header=4, quotechar='"',decimal=',')
generation = pd.read_csv('../raw_data/electricity_generation.csv',sep=',',header=4, quotechar='"',decimal=',')


## For each dataset we define the names we want to split it up: their name categories

In [21]:
categories_generation = ['Generation', 'Nuclear', 'Renewables', 'Fossil Fuels', 'Hydroelectric Pumped Storage']
categories_capacity = ['Capacity', 'Nuclear', 'Renewables', 'Fossil Fuels', 'Hydroelectric Pumped Storage']

## We save the index where the different datasets have to been splitted up

In [22]:
indexes = []
for i in categories_generation:
    indexes.append(generation[generation['Unnamed: 1'] == i].index[0])

## Slicing every dataset (from a total of 2: generation and capacity of eletrical energy) and save them in two dataframes.

In [23]:
#definition of slicing function

def slicing(df, indexes):
    dataframes = []
    for i in range(len(indexes) - 1):
        dataframes.append(df.iloc[indexes[i]: indexes[i + 1]])
    else:
        dataframes.append(df.iloc[indexes[i + 1]:])
    return dataframes

##saving the datases in a list of dataframes

generation = slicing(generation,indexes)
capacity = slicing (capacity,indexes)

### Rename all dataset (5 datasets from the 2 initial 'SUPER'dataset), makes a total of 10

In [24]:
gen_total= generation[0].reset_index(drop=True)
gen_nuclear = generation[1].reset_index(drop=True)
gen_renewable = generation[2].reset_index(drop=True) 
gen_fossil = generation[3].reset_index(drop=True) 
gen_hidro = generation[4].reset_index(drop=True) 

cap_total = capacity[0].reset_index(drop=True)
cap_nuclear = capacity[1].reset_index(drop=True)
cap_renewable = capacity[2].reset_index(drop=True)
cap_fossil = capacity[3].reset_index(drop=True)
cap_hidro = capacity[4].reset_index(drop=True)




### Read the other 4 'simple' dataset

In [25]:
consumption = pd.read_csv('../raw_data/electricity_consumption.csv',sep=',',header=4, quotechar='"',decimal=',')
exports = pd.read_csv('../raw_data/electricity_exports.csv',sep=',',header=4, quotechar='"',decimal=',')
imports = pd.read_csv('../raw_data/electricity_imports.csv',sep=',',header=4, quotechar='"',decimal=',')
population = pd.read_csv('../raw_data/population.csv',sep=',',header=4, quotechar='"',decimal=',')
GDP = pd.read_csv('../raw_data/GDP.csv',sep=',',header=4, quotechar='"',decimal=',')
C02 = pd.read_csv('../raw_data/C02emissons.csv',sep=',',header=4, quotechar='"',decimal=',')

## Gathering all useful datasets as function of how many initial rows we need to drop, because there is just the category of attribute. 

## That information it in the name of the proper dataframe

In [26]:
features_1 = [consumption, exports, imports, population, gen_total,cap_total,gen_renewable,cap_renewable,GDP,C02]
features_2 = [gen_nuclear, gen_fossil, gen_hidro, cap_nuclear,cap_fossil,cap_hidro]

---------------------------------------------------------------------------------------------

# 2nd STEP: CLEANING SUPERSET

## Define two functions: *CLEAN1* and *CLEAN2* to clean all datasets on an authomatized way.

---------------------------------------------------------------------------------------------


## CLEAN1 FUNCTION: *dataset with two first rows to drop*

### #1 dropping the empty  and the units columns

### #2 dropping the first 2 rows because there are the quantity to analyze and the category

### #3 rename the country column

### #4 Set'country' index

### #5 Replacing '--', '-' for nan. I discover for trial and error.

### #6 Drop the missing values.

### #7 Convert the data from objetct to number (float or int)

### #8  Transpose the matrix to have the country as index and year as columns.


In [27]:
def clean1(df):
    #1
    df.drop('Unnamed: 0',axis=1,inplace=True)

    df.drop('Unnamed: 2',axis=1,inplace=True)

    #2
    df.drop([0,1],axis=0,inplace=True)

    #3
    df=df.transpose()

    #4        
    header = df.iloc[0]
    df =df[1:]
    df.rename(columns =header,inplace=True)
    df.insert(0,'year', df.index)
    df.reset_index(drop =True, inplace=True)

    #5
    df.replace('--', np.nan, inplace=True)
    df.replace('-', np.nan, inplace=True)

    #6
    df.dropna(axis=1,inplace=True)

    #7
    df = df.apply(pd.to_numeric)
    return df


In [28]:
features_new1 = []
for df in features_1:
    features_new1.append(clean1(df))


## CLEAN2 FUNCTION: *dataset with just first row to drop*

### #1 dropping the empty  and the units columns

### #2 dropping the first 2 rows because there are the quantity to analyze and the category

### #3 rename the country column

### #4 Set'country' index

### #5 Replacing '--', '-' for nan. I discover for trial and error.

### #6 Drop the missing values.

### #7 Convert the data from objetct to number (float or int)

### #8  Transpose the matrix to have the country as index and year as columns.


In [29]:
def clean2(df):
    #1
    df.drop('Unnamed: 0',axis=1,inplace=True)

    df.drop('Unnamed: 2',axis=1,inplace=True)

    #2
    df.drop([0],axis=0,inplace=True)

    #3
    df=df.transpose()

    #4        
    header = df.iloc[0]
    df =df[1:]
    df.rename(columns =header,inplace=True)
    df.insert(0,'year', df.index)
    df.reset_index(drop =True, inplace=True)

    #5
    df.replace('--', np.nan, inplace=True)
    df.replace('-', np.nan, inplace=True)

    #6
    df.dropna(axis=1,inplace=True)

    #7
    df = df.apply(pd.to_numeric)
    return df


In [30]:
features_new2 = []
for df in features_2:
    features_new2.append(clean2(df))

---------------------------------------------------------------------------------------------

# 3rd Read and clean the ranking datasets
---------------------------------------------------------------------------------------------

In [31]:
ranking_consumption_2017 = pd.read_csv('../raw_data/ranking/electricity_net_consumption_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_renewable_2017 = pd.read_csv('../raw_data/ranking/renewable_electricity_net_generation_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_norenewable_2017 = pd.read_csv('../raw_data/ranking/fossil_fuels_electricity_net_generation_2017.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
ranking_CO2_emissions_2016 = pd.read_csv('../raw_data/ranking/CO2_emissions_2016.csv',sep=',',header=5, quotechar='"',decimal=',',index_col='Ranking')
rankings=list([ranking_consumption_2017, ranking_renewable_2017, ranking_norenewable_2017,ranking_CO2_emissions_2016])

In [32]:
feature_ranking = []
for item in rankings:
            item.drop('Unnamed: 0',axis=1, inplace=True)
            item['Value'] = pd.to_numeric(item['Value'],errors='coerce')
            item.dropna(inplace=True)
            feature_ranking.append(item)
            

In [33]:
features_1 = ['consumption', 'exports', 'imports', 'population', 'generation_total','capacity_total','generation_renewable','capacity_renewable','GDP','C02']
features_2 = ['generation_nuclear', 'generation_fossil', 'generation_hidrodynamic', 'capacity_nuclear','capacity_fossil','capacity_hidro']
namesranking = ['ranking_consumption_2017','ranking_renewable_2017', 'ranking_norenewable_2017','ranking_CO2_emissions_2016']


saving1 = [features_new1[i].to_csv(f"../cleaned_data/{j}.csv",index= False) for (i,j) in zip(range(len(features_new1)),features_1)]
saving2 = [features_new2[k].to_csv(f"../cleaned_data/{l}.csv",index= False) for (k,l) in zip(range(len(features_new2)),features_2)]
saving_rankining = [feature_ranking[m].to_csv(f"../cleaned_data/ranking/{n}.csv",index= False) for (m,n) in zip(range(len(feature_ranking)),namesranking)]
